In [2]:
import gzip
import json
import sys
sys.path.append(sys.path[0][:sys.path[0].find('DVML-P7') + len('DVML-P7')])

import pandas as pd
from rdflib import Namespace, Graph, URIRef, Literal, BNode

from rdflib.namespace import RDFS
from Code.UtilityFunctions.dictionary_functions import flatten_dictionary
from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.string_functions import split_words, turn_words_singular, split_words_inc_slash
from Code.UtilityFunctions.wikidata_functions import wikidata_query, retrieve_wikidata_claims, category_query, min_qid, get_all_wikidata_claims, compare_qids, categories_dict_singular
from Code.UtilityFunctions.schema_functions import get_schema_predicate, get_schema_type, get_class_mappings
from Code.UtilityFunctions.get_uri import get_uri

In [12]:
class_mappings = pd.read_csv(get_path("class_mappings.csv"))

In [13]:
class_mappings

,YelpCategory,SchemaType
0,Airport,Airport
1,Osteopath,Osteopathic
2,Repair,AutoRepair
3,RadioStation,RadioStation
4,HardwareStore,HardwareStore
...,...,...
213,Hostel,Hostel
214,EmploymentLaw,EmploymentAgency
215,Supplement,DietarySupplement
216,Trust,UKTrust


In [8]:
from shexer.shaper import Shaper
from shexer.consts import NT, SHEXC, SHACL_TURTLE

target_classes = [
    "http://example.org/Person",
    "http://example.org/Gender"
]

namespaces_dict = {"http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
                   "http://example.org/": "ex",
                   "http://weso.es/shapes/": "",
                   "http://www.w3.org/2001/XMLSchema#": "xsd"
                   }

raw_graph = """
<http://example.org/sarah> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Person> .
<http://example.org/sarah> <http://example.org/age> "30"^^<http://www.w3.org/2001/XMLSchema#int> .
<http://example.org/sarah> <http://example.org/name> "Sarah" .
<http://example.org/sarah> <http://example.org/gender> <http://example.org/Female> .
<http://example.org/sarah> <http://example.org/occupation> <http://example.org/Doctor> .
<http://example.org/sarah> <http://example.org/brother> <http://example.org/Jim> .

<http://example.org/jim> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Person> .
<http://example.org/jim> <http://example.org/age> "28"^^<http://www.w3.org/2001/XMLSchema#int> .
<http://example.org/jim> <http://example.org/name> "Jimbo".
<http://example.org/jim> <http://example.org/surname> "Mendes".
<http://example.org/jim> <http://example.org/gender> <http://example.org/Male> .

<http://example.org/Male> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Gender> .
<http://example.org/Male> <http://www.w3.org/2000/01/rdf-schema#label> "Male" .
<http://example.org/Female> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Gender> .
<http://example.org/Female> <http://www.w3.org/2000/01/rdf-schema#label> "Female" .
<http://example.org/Other> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Gender> .
<http://example.org/Other> <http://www.w3.org/2000/01/rdf-schema#label> "Other gender" .
"""



input_nt_file = "target_graph.nt"

shaper = Shaper(target_classes=target_classes,
                graph_file_input=input_nt_file,
                input_format=NT,
                namespaces_dict=namespaces_dict,  # Default: no prefixes
                instantiation_property="http://www.w3.org/1999/02/22-rdf-syntax-ns#type")  # Default rdf:type

output_file = "shaper_example.shex"

shaper.shex_graph(output_file=output_file,
                  acceptance_threshold=0.)

print("Done!")

Done!


In [10]:
set([1,2,3]).difference(set([1,2]))

{3}

In [11]:
schema_category_mappings_df = pd.read_csv(get_path("class_mappings.csv"))
schema_category_mappings_dict = dict([(i, eval(x)) for i, x in zip(schema_category_mappings_df['YelpCategory'],schema_category_mappings_df['SchemaType'])])

In [25]:
category = "Art_Classes"
if category in schema_category_mappings_dict.keys():
    print(category)